# evaluomeR Step 3

### Step 3 <a class="anchor" id="step_3"></a>

In [1]:
library("evaluomeR")
library("psych")
library("scales")
library("caret")

options(scipen=10)

Loading required package: SummarizedExperiment
Loading required package: GenomicRanges
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: 'BiocGenerics'

The following objects are masked from 'package:parallel':

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs

The following objects are masked from 'package:base':

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which

In [2]:
seed = 13606
k.range=c(2,6)
set.seed(seed)
input_df = read.csv(paste0(getwd(), "/","../../usecases/usecase2/data/agro.csv"), header=TRUE, stringsAsFactors=FALSE)

In [3]:
evaluomeRSupportedCBI()

[1] "kmeans"    "clara"     "clara_pam" "hclust"    "pamk"      "pamk_pam" 
[7] "rskc"

In [8]:
# Calculamos el L1 previamente para cada K, así el cómputo de evaluomeR es más rápido.
data = input_df[-1]
head(data)
for (k in k.range[1]:k.range[2]) {
    wbounds = seq(2,sqrt(ncol(data)), len=30)
    km.perm <- sparcl::KMeansSparseCluster.permute(data,K=k,wbounds=wbounds,nperms=5,silent=TRUE)
    L1 = km.perm$bestw
    print(paste0("Best L1 for k=", k, " is: ", L1))
}

ANOnto,AROnto,CBOOnto,CBOOnto2,CROnto,DITOnto,INROnto,LCOMOnto,NACOnto,NOCOnto,NOMOnto,POnto,PROnto,RFCOnto,RROnto,TMOnto,TMOnto2,WMCOnto,WMCOnto2
0.0000000,3.9503849,0.9991446,0.9991446,0.9957228,3,0.9991446,1.999142,1.000000,292.000000,2.9632164,0.9957228,0.7478411,3.9623610,0.252158895,0.00000000,0.000000,1.999142,1.000000
0.9298246,0.5438596,0.9824561,0.9824561,0.0000000,5,0.9824561,2.357143,1.000000,3.733333,0.9824561,0.5789474,0.5000000,1.9649123,0.500000000,0.00000000,0.000000,2.357143,1.000000
0.7500000,0.0000000,0.8750000,0.8750000,3998.8750000,3,0.8750000,1.333333,1.000000,3.500000,2275.7500000,0.2500000,0.9996157,2276.6250000,0.000384341,0.00000000,0.000000,1.333333,1.000000
0.9907407,3.1018519,1.0694444,1.0694444,0.3634259,16,1.0694444,7.695971,1.052174,2.287129,1.2037037,1.0555556,0.5295316,2.2731481,0.470468432,0.06264501,2.148148,9.134783,1.186957
1.2362637,0.0000000,1.0659341,1.0659341,0.0000000,6,1.0329670,2.467532,1.077465,4.700000,0.5879121,0.8571429,0.3627119,1.6538462,0.637288136,0.07182320,2.000000,2.676056,1.084507
0.0000000,0.0000000,0.6666667,0.6666667,1107.6666670,2,0.6666667,1.000000,1.000000,2.000000,0.0000000,0.0000000,0.0000000,0.6666667,1.000000000,0.00000000,0.000000,1.000000,1.000000


[1] "Best L1 for k '2' is: 2"
[1] "Best L1 for k '3' is: 2"
[1] "Best L1 for k '4' is: 2"
[1] "Best L1 for k '5' is: 2"
[1] "Best L1 for k '6' is: 2"


In [19]:
# Create a structure
rskc_run <- function(stab, qual, alpha) {
  structure(list(stab = stab, qual = qual, alpha = alpha), class = "rskc_run")
}
run_list = list()

In [38]:
RSKC(data, 3, alpha=0.24, L1 = 2)


Input: 
#obs= 78  #feature= 19 
L1= 2  alpha= 0.24

Result:
wbss: 1265.826
#non-zero weights: 19 
 3 clusters of sizes 24, 25, 29 
Cluster labels: 2 1 1 3 1 1 1 3 3 1 1 1 3 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 1 1 1 1 3 3 3 3 3 1 1 1 3 2 1 1 3 3 1 2 3 2 1 3 3 3 3 3 1 3 3 3 3 3 3 1 3 3 3 3 3 

In [47]:
alpha_values = seq(0, 0.25, 0.05)
index = 1
for (alpha in alpha_values) {
    stab = stabilityRange(data=input_df, k.range=k.range, 
                                bs=100, seed=seed,
                                all_metrics=TRUE,
                                cbi="rskc", L1=2, alpha=alpha)
    stab_table = standardizeStabilityData(stab)
    
    qual = qualityRange(data=input_df, k.range=k.range, 
                                seed=seed,
                                all_metrics=TRUE,
                                cbi="rskc", L1=2, alpha=alpha)
    qual_table = standardizeQualityData(qual)
    
    run_list[[index]] <- rskc_run(stab = stab_table, qual = qual_table, alpha=alpha)
    index = index + 1
}


Data loaded.
Number of rows: 78
Number of columns: 20


Processing all metrics, 'merge', in dataframe (19)
	Calculation of k = 2
	Calculation of k = 3
	Calculation of k = 4
	Calculation of k = 5
	Calculation of k = 6

Data loaded.
Number of rows: 78
Number of columns: 20


Processing all metrics, 'merge', in dataframe (19)
	Calculation of k = 2
	Calculation of k = 3
	Calculation of k = 4
	Calculation of k = 5
	Calculation of k = 6

Data loaded.
Number of rows: 78
Number of columns: 20


Processing all metrics, 'merge', in dataframe (19)
	Calculation of k = 2


alpha is too large for n 
alpha is too large for n 
alpha is too large for n 
alpha is too large for n 
alpha is too large for n 
alpha is too large for n 
alpha is too large for n 
alpha is too large for n 
alpha is too large for n 
Step (a), (a2) and (b) are repeated over the maximum number of iterations. The algorithm might not converge.alpha is too large for n 
alpha is too large for n 
alpha is too large for n 
alpha is too large for n 
alpha is too large for n 
alpha is too large for n 
alpha is too large for n 
alpha is too large for n 
alpha is too large for n 
Step (a), (a2) and (b) are repeated over the maximum number of iterations. The algorithm might not converge.alpha is too large for n 
alpha is too large for n 
alpha is too large for n 
alpha is too large for n 
alpha is too large for n 
alpha is too large for n 
alpha is too large for n 
alpha is too large for n 
alpha is too large for n 
Step (a), (a2) and (b) are repeated over the maximum number of iterations. The alg

	Calculation of k = 3
	Calculation of k = 4
	Calculation of k = 5
	Calculation of k = 6

Data loaded.
Number of rows: 78
Number of columns: 20


Processing all metrics, 'merge', in dataframe (19)
	Calculation of k = 2
	Calculation of k = 3
	Calculation of k = 4
	Calculation of k = 5
	Calculation of k = 6

Data loaded.
Number of rows: 78
Number of columns: 20


Processing all metrics, 'merge', in dataframe (19)
	Calculation of k = 2
	Calculation of k = 3
	Calculation of k = 4
	Calculation of k = 5
	Calculation of k = 6

Data loaded.
Number of rows: 78
Number of columns: 20


Processing all metrics, 'merge', in dataframe (19)
	Calculation of k = 2
	Calculation of k = 3
	Calculation of k = 4
	Calculation of k = 5
	Calculation of k = 6

Data loaded.
Number of rows: 78
Number of columns: 20


Processing all metrics, 'merge', in dataframe (19)
	Calculation of k = 2
	Calculation of k = 3
	Calculation of k = 4
	Calculation of k = 5
	Calculation of k = 6

Data loaded.
Number of rows: 78
Number 

In [50]:
run_list

[[1]]
$stab
                 k_2      k_3       k_4       k_5       k_6
all_metrics 0.922938 0.763779 0.6835521 0.6574005 0.7228951

$qual
                  k_2       k_3       k_4       k_5        k_6
all_metrics 0.9765144 0.9652177 0.9491769 0.9192709 0.09758856

$alpha
[1] 0

attr(,"class")
[1] "rskc_run"

[[2]]
$stab
                  k_2       k_3      k_4       k_5       k_6
all_metrics 0.7117527 0.5596798 0.509575 0.6739369 0.6278424

$qual
                  k_2       k_3       k_4        k_5        k_6
all_metrics 0.9293708 0.8471999 0.8209064 0.06414705 -0.1134065

$alpha
[1] 0.05

attr(,"class")
[1] "rskc_run"

[[3]]
$stab
                 k_2       k_3       k_4       k_5       k_6
all_metrics 0.796572 0.6348133 0.4021418 0.4315106 0.5291065

$qual
                   k_2        k_3        k_4        k_5        k_6
all_metrics -0.6832273 -0.4322093 -0.4406368 -0.4011023 0.04653789

$alpha
[1] 0.1

attr(,"class")
[1] "rskc_run"

[[4]]
$stab
                  k_2       k_3     